# Task 1

%
If you haven't already done so, the first task is to form a group of 2-4 people. There is an announcement on Absalon describing how to do this. Make sure that you list the names of all members of the group at the top of the jupyter notebook along with your group number.
%

## Group members:
* Oleksandr kryshtalov [mdq842]  
* Ulrik Bjørn Meelby [na]  


# Task 2

%
For our fake news predictor, we will be using the FakeNewsCorpus dataset as our primary dataset. 
It is available from this github repository:, where you can also find information about how the data is collected, the available fields, etc. In this first milestone, we will work only on a small subset of the FakeNewsCorpus dataset. 
Your first task is to retrieve this subset from https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv and structure/process/clean it. Describe which procedures (and which libraries) you used and why they are appropriate.
%

In order to clean the data we need to first know what the primary goal of cleaning it is. 
In task 3 we have chosen to analyze following questions:
* Are there words that have a higher frequency in fake-news articles over trustworthy ones, and what are those?
* What are the meta_keywords that have the highest inclination to be fake?
* Observe zipf law on english text with given dataset.

All observations require text tokenization, and a way to clean and stem them, in order to not differentiate between words, eg. word and words.

For the cleaning of the text we will be using cleatext.
nltk will be used for tokinization via word_tokenize, and for stemming via the porter stemmer.

In [1]:
# imports:
import requests
import re
import numpy as np
import pandas as pd
import string

from cleantext import clean
import nltk
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [2]:
#We get the data from the web 
csvdata = requests.get("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv")
csvdata

<Response [200]>

In [3]:
# Write csvfile
with open("rawdata.csv","wb") as file:
    file.write(csvdata.content)

# Parse csv into dataframe
dfRaw = pd.read_csv ('rawdata.csv')

# drop some tables for brevity:
df = dfRaw.iloc[: , 1:]
df = df.drop(columns =[ "url", "scraped_at", "scraped_at", "inserted_at", "updated_at", "keywords", "summary"])

# have a peek at the data:
#df

## Cleaning:

In [4]:
# We want to make a pd.Series -> pd.Series function that cleans all the text
# lowercases everything removes urls, emails etc.

def clean_text(textSeries:pd.Series) -> pd.Series:
    def clean_func (text):
        return clean(text,
            fix_unicode=True,            
            to_ascii=True,               
            lower=True,                  
            no_line_breaks=True,        
            no_urls=True,                
            no_emails=True,              
            no_phone_numbers=False,      
            no_numbers=True,             
            no_punct=True,              
            replace_with_punct="",       
            replace_with_url="oURL", # clean_text lowers after tagging for some reason???
            replace_with_email="oEMAIL",
            replace_with_phone_number="oPHONE",
            replace_with_number="oNUM",
            replace_with_digit="0",
            lang="en")
    return textSeries.apply(clean_func)
            
df.content = clean_text(df.content)

## Tokenization:

In [5]:
nltk.download('punkt') # fix

# Same as before a pd.Series -> pd.Series function.
def tokenize(textSeries:pd.Series) -> pd.Series:
    return pd.Series(textSeries.apply(nltk.tokenize.wordpunct_tokenize),name="tokens")
df["tokens"] = tokenize(df.content)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Oleks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Removal of stopwords:

## Stemming:

In [7]:
# stems all the tokens
def stem (data:pd.Series)-> pd.Series:
    ps = PorterStemmer()
    # function to be applied on each element in series
    def stem (tokens):
        return [ps.stem(token) for token in tokens]
    return pd.Series(data.apply(stem))

# removal of duplicates
def dupeRemove (tokens:pd.Series) -> pd.Series:
    def unDupe (tokens):
        return list(set(tokens))
    return tokens.apply(unDupe)



df["stemmed_with_dupes"] = stem(df.tokens)
df["stemmed"] = dupeRemove(df.tokens)

# Task 3
%\
Now try to explore the FakeNewsCorpus dataset. Make at least three non-trivial observations/discoveries about the data. These observations could be related to outliers, artefacts, or even better: genuinely interesting patterns in the data that could potentially be used for fake-news detection. Examples of simple observations could be how many missing values there are in particular columns - or what the distribution over domains is. Be creative! :).  
%

We found some observations we wanted to look into in Task 2: 
1. Are there words that have a higher frequency in fake-news articles over trustworthy ones, and what are those?
2. What are the meta_keywords that have the highest inclination to be fake?
3. Observe zipf law on english text with given dataset.

These tasks can then be split into smaller tasks:
### 1


### 2
Find all keywords \
Iterate over each fake/credible adding to each each frequency counter.\


## 3
Create a vocabulary for the entire dataset \
calculate bag of words for all text. \
Sort BOW by frequency. \
plot on a scatterplot. 
